[View in Colaboratory](https://colab.research.google.com/github/gmihaila/machine_learning_toolbox/blob/master/kera_multi_gpu.ipynb)

# Multi GPU Keras training
## This is for 2 or more gpus, by default keras uses 1 gpu if installed


In [1]:
import os, sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

os.environ["CUDA_VISIBLE_DEVICES"]="2,3"  #<------ CHECK YOUR MLP EMAIL

from keras.utils.training_utils import multi_gpu_model #<------------------Esential for multi GPUs usage


# BUILD MODEL FUNCTION

def BuildToyModel():
    # generate some dummy data
    n_examples = 1000
    n_features = 100

    x_train = np.random.random((n_examples, n_features))
    x_train = np.round(x_train, 2)
    x_train *= 100
    x_train = np.array(x_train, dtype=int)

    n_words = np.max(x_train)

    y_train = np.random.random(n_examples)

    # number of units in LSTM
    n_units = 256
    # numbe rof words
    n_words = 100500   #vocabulary size
    size_emb = 300     #size of embedding has to match
    size_seq = 50

    embedding_matrix = np.random.random((n_words+2, size_emb))

    # build model
    model = Sequential()
    model.add(Embedding(input_dim=(n_words+2), output_dim=size_emb, weights=[embedding_matrix],
                    mask_zero=False, trainable=False))
    # LSTM LAYER/S
    model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))

    # TREAT AS REGRESSION PROBLEM
    model.add(Dense(1, activation='sigmoid'))

    return model, x_train, y_train
  

Using TensorFlow backend.


In [0]:
# Build model
model, x_train, y_train = BuildToyModel()

# Add how many GPUs you want/have:wq
model = multi_gpu_model(model, gpus=1)          #<------------------Esential Part to run on GPUs

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])

# model summary
print(model.summary())                          #<------------------See lambda layers distributed on each GPU

# batches
n_batch = 32       #<------------------make sure your batch is not too small
# epochs
n_epoch = 10

# train model
model.fit(x_train, y_train, epochs=n_epoch, batch_size=n_batch, verbose=1)